In [1]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

Get:1 http://security.ubuntu.com/ubuntu bionic-security InRelease [88.7 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu bionic-cran40/ InRelease [3,626 B]
Ign:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  InRelease
Ign:4 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  InRelease
Get:5 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release [696 B]
Get:6 http://ppa.launchpad.net/c2d4u.team/c2d4u4.0+/ubuntu bionic InRelease [15.9 kB]
Hit:7 https://developer.download.nvidia.com/compute/machine-learning/repos/ubuntu1804/x86_64  Release
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu1804/x86_64  Release.gpg [836 B]
Hit:9 http://archive.ubuntu.com/ubuntu bionic InRelease
Get:10 http://archive.ubuntu.com/ubuntu bionic-updates InRelease [88.7 kB]
Hit:11 http://ppa.launchpad.net/cran/libgit2/ubuntu bionic InRelease
Get:12 http://archive.ubuntu.com/ubuntu bionic-backpor

In [2]:
# Initiate the Spark Session
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("VineAnalysis").getOrCreate()

In [3]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_PC_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_PC_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|marketplace|customer_id|     review_id|product_id|product_parent|       product_title|product_category|star_rating|helpful_votes|total_votes|vine|verified_purchase|     review_headline|         review_body|review_date|
+-----------+-----------+--------------+----------+--------------+--------------------+----------------+-----------+-------------+-----------+----+-----------------+--------------------+--------------------+-----------+
|         US|   22873041|R3ARRMDEGED8RD|B00KJWQIIC|     335625766|Plemo 14-Inch Lap...|              PC|          5|            0|          0|   N|                Y|Pleasantly surprised|I was very surpri...| 2015-08-31|
|         US|   30088427| RQ28TSA020Y6J|B013ALA9LA|     671157305|TP-Link OnHub AC1...|              PC|          5|    

In [6]:
# Create the vine_table DataFrame
vine_df = df.select(["review_id", "star_rating", "helpful_votes", "total_votes", "vine", "verified_purchase"])
vine_df.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3ARRMDEGED8RD|          5|            0|          0|   N|                Y|
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
| RUXJRZCT6953M|          1|            2|          2|   N|                N|
| R7EO0UO6BPB71|          1|            0|          0|   N|                Y|
|R39NJY2YJ1JFSV|          5|            0|          0|   N|                Y|
|R31SR7REWNX7CF|          5|            0|          0|   N|                Y|
| RVBP8I1R0CTZ8|          3|            1|          2|   N|                Y|
|R1QF6RS1PDLU18|          4|            1|          1|   N|                Y|
|R23AICGEDAJQL1|          1|            0|          0|   N|                Y|
|R2EY3N4K9W19UP|          5|            3|          4|   N|     

In [7]:
# Step 1: Filter the vine_df
vine_df_filtered = vine_df.filter("total_votes>=20")
vine_df_filtered.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R35OW0SJPJWAHO|          1|           12|        162|   N|                Y|
| RAP2ETU511BOI|          1|            5|         99|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R1RMOCWMV8F4VH|          1|            5|         21|   N|                Y|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|     

In [9]:
# Step 2: Create DF to retrieve all the rows where the number of helpful_votes divided by total_votes is equal to or greater than 50%
vine_df_helpful = vine_df_filtered.filter((vine_df_filtered["helpful_votes"])/(vine_df_filtered["total_votes"]) >= 0.50)
vine_df_helpful.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|
|R2GK7RJNIAREIT|          1|           30|         32|   N|                Y|
|R2OWRP6FXXECMK|          1|           42|         44|   N|                Y|
| ROR2FE6Y8EZYG|          5|           19|         22|   N|     

In [11]:
# Step 3: create a new DataFrame or table that retrieves all the rows where a review was written as part of the Vine program (paid), vine == 'Y'.
vine_df_yes = vine_df_helpful.filter(vine_df_helpful["vine"] == "Y")
vine_df_yes.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
|R3J2DPYUH6453D|          5|          136|        153|   Y|                N|
|R25QTFLERWK6SJ|          3|          852|        956|   Y|                N|
|R1BG71K9B61LI1|          4|           39|         39|   Y|                N|
| R8WGX426U56E0|          5|           19|         21|   Y|                N|
| R614TCIDD36LE|          4|          131|        147|   Y|                N|
|R28X5K1OZBQSBK|          5|           25|         28|   Y|                N|
|R2CUDWKAUWGHIT|          5|           19|         24|   Y|                N|
| R886ADSI1HBU5|          5|           88|         97|   Y|                N|
|R3TTJTAND1CCSV|          5|           21|         23|   Y|                N|
|R1W39WXYWNFJBG|          5|           32|         35|   Y|     

In [12]:
# Step 4: create DataFrame limited to vine == "N"
vine_df_no = vine_df_helpful.filter(vine_df_helpful["vine"] == "N")
vine_df_no.show()

+--------------+-----------+-------------+-----------+----+-----------------+
|     review_id|star_rating|helpful_votes|total_votes|vine|verified_purchase|
+--------------+-----------+-------------+-----------+----+-----------------+
| RQ28TSA020Y6J|          5|           24|         31|   N|                N|
|R25D83AEJRXUWO|          4|           18|         21|   N|                Y|
|R1S9O5CCTD61ZD|          1|           45|         71|   N|                N|
|R351KVE3VO78W9|          3|           36|         42|   N|                Y|
| R4LANIHO6EXAH|          4|           35|         35|   N|                Y|
|R3V26MZUOWAUA4|          3|           35|         45|   N|                Y|
|R38OLNY5GR1PW6|          5|           44|         44|   N|                Y|
|R2GK7RJNIAREIT|          1|           30|         32|   N|                Y|
|R2OWRP6FXXECMK|          1|           42|         44|   N|                Y|
| ROR2FE6Y8EZYG|          5|           19|         22|   N|     

In [21]:
# Step 5: Determine summary statistics
vine_yes_total = vine_df_yes.count()
vine_yes_fivestar = vine_df_yes.filter("star_rating=5").count()
vine_yes_percentage = (vine_yes_fivestar / vine_yes_total) * 100

vine_no_total = vine_df_no.count()
vine_no_fivestar = vine_df_no.filter("star_rating=5").count()
vine_no_percentage = (vine_no_fivestar / vine_no_total) * 100

summary_df = spark.createDataFrame([("Yes", vine_yes_total, vine_yes_fivestar, vine_yes_percentage), ("No", vine_no_total, vine_no_fivestar, vine_no_percentage)], ["Vine Review", "Total Reviews", "Five Star Reviews", "% Five Star"])

summary_df.show()

+-----------+-------------+-----------------+------------------+
|Vine Review|Total Reviews|Five Star Reviews|       % Five Star|
+-----------+-------------+-----------------+------------------+
|        Yes|         1775|              783| 44.11267605633803|
|         No|        77385|            35949|46.454739290560184|
+-----------+-------------+-----------------+------------------+

